

# 📘 SafeSportAdvisor Backend – Code Explanation (Markdown)

## 1. Overview

This backend is built using **Node.js** and **Express.js**.
It powers the *SafeSportAdvisor* application by intelligently routing user queries to either a **local LLM (Ollama)** or an **external LLM (OpenAI)** while prioritizing **privacy, cost efficiency, and reliability**.

### Core Responsibilities

* Receive user messages from the frontend
* Classify questions as *easy* or *complex*
* Anonymize sensitive data when necessary
* Route requests to the appropriate LLM
* Return responses with metadata

---

## 2. Project Setup & Dependencies

```js
require('dotenv').config();
const express = require('express');
const axios = require('axios');
```

### Explanation

* **dotenv**: Loads environment variables from `.env`
* **express**: Web framework for building APIs
* **axios**: HTTP client for calling LLM APIs

---

## 3. Server Initialization

```js
const app = express();
const port = process.env.PORT || 3000;
```

* Initializes the Express app
* Uses a configurable port (default: `3000`)

---

## 4. Middleware Configuration

### JSON Parsing

```js
app.use(express.json());
```

Allows the server to read JSON request bodies.

---

### CORS Configuration

```js
app.use((req, res, next) => {
  res.header('Access-Control-Allow-Origin', '*');
  res.header('Access-Control-Allow-Headers', 'Content-Type');
  next();
});
```

**Purpose**:

* Allows frontend requests from any origin
* Required when opening `index.html` using `file://`

---

## 5. Main API Endpoint: `/chat`

```js
app.post('/chat', async (req, res) => {
```

This endpoint processes user messages and returns advisor responses.

---

### Input Validation

```js
if (!message || typeof message !== 'string') {
  return res.status(400).json({ error: 'Message is required' });
}
```

Ensures the request contains a valid message.

---

## 6. Intelligent Question Routing

### Easy Question Detection

```js
const isEasy = await isQuestionEasy(message);
```

An LLM-based classifier determines whether:

* The question can be answered locally
* External knowledge is required

---

## 7. Easy Question Flow (Local LLM)

```js
reply = await callLocalLLM(prompt);
```

**Characteristics**:

* Uses Ollama (local model)
* No anonymization
* Faster and privacy-preserving

---

## 8. Complex Question Flow (Privacy-First)

### Step 1: Anonymization

```js
const safeMessage = await anonymizeMessage(message);
```

Sensitive data replaced with placeholders such as:

* `[AGE]`
* `[HEART_RATE]`
* `[NAME]`

---

### Step 2: LLM Selection

```js
if (process.env.OPENAI_API_KEY) {
  reply = await callExternalLLM(prompt);
} else {
  reply = await callLocalLLM(prompt);
}
```

**Fallback strategy** ensures system reliability.

---

## 9. API Response Format

```js
res.json({
  reply,
  routed_to: llmUsed,
  anonymized: !isEasy
});
```

### Response Fields

* **reply**: Advisor’s response
* **routed_to**: `local` or `external`
* **anonymized**: Boolean flag

---

## 10. Error Handling

```js
catch (err) {
  res.status(500).json({
    error: 'Advisor unavailable. Check if Ollama is running.'
  });
}
```

Gracefully handles failures without crashing the server.

---

## 11. Helper Functions

### `callLocalLLM(prompt)`

* Sends requests to Ollama (`localhost:11434`)
* Uses temperature and token limits for balanced output

---

### `callExternalLLM(prompt)`

* Calls OpenAI Chat Completion API
* Requires `OPENAI_API_KEY`
* Only receives anonymized input

---

### `anonymizeMessage(text)`

* Uses LLM-based redaction
* Removes personal, biometric, and medical data
* Preserves sports/medical context

---

### `isQuestionEasy(text)`

* LLM-based router
* Returns **yes/no** only
* Optimizes cost and privacy

---

## 12. Server Startup

```js
app.listen(port, () => {
  console.log(`Safe Sport Advisor backend running on http://localhost:${port}`);
});
```

---

## 13. Architecture Summary

**Design strengths**:

* 🧠 Smart LLM routing
* 🔒 Privacy-first anonymization
* 💰 Cost-efficient API usage
* 🔁 Robust fallbacks
* 🏠 Local-first AI approach

